In [1]:
# Imports das bilbiotecas necessárias
import pandas as pd
import numpy as np

ANÁLISE EXPLORATÓRIA DO DATASET

In [2]:
# Analisando o Dataset
dfpath = "D:\\FACULDADE\\SEGUNDO SEMESTRE\\ALGEBRA LINEAR\\PCA\\googleplaystore.csv"
df = pd.read_csv(dfpath)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [3]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Colunas úteis: Rating, Reviews, Size, Installs, Type, Price, Content Rating

Colunas desnecessárias: App, Last Updated, Current Ver, Android Ver, Genres, Category

Colunas que necessitam de conversão: Type, Content Rating

In [4]:
# Removendo colunas desnecessárias
D = df.drop(columns=['App', 'Last Updated', 'Current Ver', 'Android Ver', 'Genres', 'Category'])

In [5]:
# Tratando a coluna 'Rating'
D['Rating'] = D['Rating'].astype(str)
D.drop(D[D['Rating'].str.strip() == 'nan'].index, inplace=True)
D['Rating'] = D['Rating'].astype(float)

D['Rating']

0        4.1
1        3.9
2        4.7
3        4.5
4        4.3
        ... 
10834    4.0
10836    4.5
10837    5.0
10839    4.5
10840    4.5
Name: Rating, Length: 9367, dtype: float64

In [6]:
# Tratando a coluna 'Reviwes'
D.loc[D['Reviews'] == "3.0M", 'Reviews'] = 3000000
D['Reviews'].astype(int)

D['Reviews']

0           159
1           967
2         87510
3        215644
4           967
          ...  
10834         7
10836        38
10837         4
10839       114
10840    398307
Name: Reviews, Length: 9367, dtype: object

In [7]:
# Tratando a coluna 'Size'
def ToMega(size):
    if isinstance(size, str):
        size = size.strip()

    if 'M' in size:
        return float(size.replace('M', ''))
    
    elif 'k' in size:
        return float(size.replace('k', '')) / 1000 
    
    return float('nan')

D['Size'] = D['Size'].apply(ToMega)
D.dropna(subset=['Size'], inplace=True)

D['Size']

0        19.000
1        14.000
2         8.700
3        25.000
4         2.800
          ...  
10833     0.619
10834     2.600
10836    53.000
10837     3.600
10840    19.000
Name: Size, Length: 7729, dtype: float64

In [8]:
# Tratando a coluna 'Installs"
D['Installs'] = D['Installs'].str.replace('+', '')
D['Installs'] = D['Installs'].str.replace(',', '')
D['Installs'] = D['Installs'].astype(int)

D['Installs']

0           10000
1          500000
2         5000000
3        50000000
4          100000
           ...   
10833        1000
10834         500
10836        5000
10837         100
10840    10000000
Name: Installs, Length: 7729, dtype: int32

In [9]:
# Tratando a coluna 'Type'
D = pd.get_dummies(D, columns=['Type'], drop_first=True) # One-Hot Enconding
D['Type_Paid'] = D['Type_Paid'].astype(int)
D.rename(columns={'Type_Paid': 'Is_Paid'}, inplace=True)

D['Is_Paid'].unique()

array([0, 1])

In [10]:
# Tratando a coluna 'Price'
D['Price'] = D['Price'].str.replace('$', '').astype(float)

np.dtype(D['Price'])

dtype('float64')

In [11]:
# Tratando a coluna 'Content Rating'
order = {
    'Everyone': 1,
    'Teen': 12,
    'Everyone 10+': 10,
    'Mature 17+': 17,
    'Adults only 18+': 18,
    'Unrated': 0
}

D['Content Rating'] = D['Content Rating'].map(order) # Label 

D['Content Rating'].unique()

array([ 1, 12, 10, 17, 18,  0], dtype=int64)

In [12]:
D = D.astype(int)

D.dtypes

Rating            int32
Reviews           int32
Size              int32
Installs          int32
Price             int32
Content Rating    int32
Is_Paid           int32
dtype: object

SAVE DO DATASET TRATADO

In [13]:
pd.DataFrame.to_csv(D, 'D:\\FACULDADE\\SEGUNDO SEMESTRE\\ALGEBRA LINEAR\\PCA\\PCA_DATASET.csv')